<a href="https://colab.research.google.com/github/claude9493/20Fall_STAT7008/blob/main/Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup